### Import libraries and functions

In [1]:
from vis_utils import VolumeVisualizer, ColorMapVisualizer, draw_graph_on_model
from identification import get_vessel_graph, remove_graph_components, parametrize_graph, clean_data_graph

In [2]:
import numpy as np
import networkx as nx

from queue import SimpleQueue
from scipy.ndimage import zoom, distance_transform_edt

### Flags

In [30]:
visualize_steps = True 

### Utils

In [4]:
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()

### Read data

In [5]:
volume = np.fromfile('../data/P13/data.raw', dtype=np.uint8)
volume = volume.reshape(877, 488, 1132)

### Cut image

In [6]:
volume = volume[200:700, :, 100:650]
val = 20

volume[:, 200:, -35:] = val
volume[-150:, 200:, -100:-35] = val
volume[:, -200:-150, -100:-35] = val
volume[:, :210, -50:] = val
volume[:150, :150, -85:-50] = val

volume = volume[-170:-40,200:480,30:260]
volume[-90:, :, -110:] = val
volume[:, -80:, -92:] = val

In [7]:
volume.shape

(130, 280, 230)

### Segmentation

In [8]:
mask = volume > 32

In [9]:
if visualize_steps:
    VolumeVisualizer(mask, binary=True).visualize() 

### Scaling

In [10]:
scaled_mask = zoom(mask, zoom=0.7, order=0)

In [11]:
if visualize_steps:
    VolumeVisualizer(scaled_mask, binary=True).visualize() 

In [12]:
scaled_mask.shape

(91, 196, 161)

### EDT transform

In [13]:
%%time
edt_img = distance_transform_edt(scaled_mask)

CPU times: total: 484 ms
Wall time: 491 ms


In [14]:
if visualize_steps:
    ColorMapVisualizer(edt_img.astype(np.uint8)).visualize()

In [15]:
%%time
vessel_graph = get_vessel_graph(scaled_mask)

Starting points:  4510
i: 0 j: 0 1


C:\Users\Alicja\anaconda3\envs\im\lib\site-packages\skimage\_shared\utils.py:157: RuntimeWarning: When min_distance < 1, peak_local_max acts as finding image > max(threshold_abs, threshold_rel * max(image)).
  return func(*args, **kwargs)


i: 0 j: 10 5
i: 0 j: 20 6
i: 0 j: 30 7
i: 0 j: 40 9
i: 0 j: 50 12
i: 0 j: 60 15
i: 0 j: 70 20
i: 0 j: 80 23
i: 0 j: 90 29
i: 0 j: 100 35
i: 0 j: 110 35
i: 0 j: 120 38
i: 0 j: 130 36
i: 0 j: 140 37
i: 0 j: 150 34
i: 0 j: 160 38
i: 0 j: 170 40
i: 0 j: 180 45
i: 0 j: 190 48
i: 0 j: 200 48
i: 0 j: 210 52
i: 0 j: 220 50
i: 0 j: 230 40
i: 0 j: 240 36
i: 0 j: 250 41
i: 0 j: 260 43
i: 0 j: 270 47
i: 0 j: 280 50
i: 0 j: 290 52
i: 0 j: 300 54
i: 0 j: 310 55
i: 0 j: 320 51
i: 0 j: 330 46
i: 0 j: 340 36
i: 0 j: 350 29
i: 0 j: 360 23
i: 0 j: 370 17
i: 0 j: 380 14
i: 0 j: 390 18
i: 0 j: 400 26
i: 0 j: 410 28
i: 0 j: 420 32
i: 0 j: 430 34
i: 0 j: 440 36
i: 0 j: 450 32
i: 0 j: 460 39
i: 0 j: 470 45
i: 0 j: 480 46
i: 0 j: 490 48
i: 0 j: 500 52
i: 0 j: 510 56
i: 0 j: 520 60
i: 0 j: 530 55
i: 0 j: 540 56
i: 0 j: 550 58
i: 0 j: 560 67
i: 0 j: 570 70
i: 0 j: 580 74
i: 0 j: 590 75
i: 0 j: 600 70
i: 0 j: 610 74
i: 0 j: 620 81
i: 0 j: 630 84
i: 0 j: 640 82
i: 0 j: 650 82
i: 0 j: 660 86
i: 0 j: 670 87
i: 0 j: 

In [16]:
print('Number of nodes', len(vessel_graph.nodes))
print('Number of edges', len(vessel_graph.edges))
print('Average degree', sum(dict(vessel_graph.degree).values()) / len(vessel_graph.nodes))

Number of nodes 4905
Number of edges 1326
Average degree 0.5406727828746177


In [17]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Remove components

In [18]:
%%time
vessel_graph_rm = remove_graph_components(vessel_graph)

CPU times: total: 15.6 ms
Wall time: 15.6 ms


In [19]:
print('Number of nodes', len(vessel_graph_rm.nodes))
print('Number of edges', len(vessel_graph_rm.edges))
print('Average degree', sum(dict(vessel_graph_rm.degree).values()) / len(vessel_graph_rm.nodes))

Number of nodes 1366
Number of edges 1326
Average degree 1.9414348462664714


In [29]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_rm)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Connect endings 3D

In [21]:
def connect_endings_3d(graph, edt_img, multiplier=2.5):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree == 1]
    
    # for every ending run BFS connection search
    i = 0
    print("Endings: ", len(endings))
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = []
        
        # run BFS on a restricted area
        j = 0
        while not points.empty() and search_area:
            if j % 10 == 0:
                print(f"i: {i} j: {j} points: ", points.qsize())
            search_area -= 1
            # get point
            x, y, z = points.get()
            visited.append((x, y, z))

            # check if point is a node and is a valid connection
            if graph.has_node((x, y, z)) and not nx.has_path(graph, start, (x, y, z)):
                graph.add_edge(start, (x, y, z))
                # this is to prevent accidentally creating bifurcations
                if (x, y, z) in endings:
                    endings.pop(endings.index((x, y, z)))
                break

            # add point to search if it is in segmentation mask and it is not visited
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    for dz in range(-1, 2):
                        new_point = (x + dx, y + dy, z + dz)
                        if (
                            x + dx >= 0
                            and x + dx < edt_img.shape[0]
                            and y + dy >= 0
                            and y + dy < edt_img.shape[1]
                            and z + dz >= 0
                            and z + dz < edt_img.shape[2]
                            and new_point not in visited
                            and edt_img[x + dx, y + dy, z + dz] > 0
                        ):
                            visited.append(new_point)
                            points.put(new_point)
            j += 1
        i += 1

    return graph

In [22]:
%%time
vessel_graph_cn = connect_endings_3d(vessel_graph_rm, edt_img, 3)

Endings:  425
i: 0 j: 0 points:  1
i: 0 j: 10 points:  38
i: 0 j: 20 points:  43
i: 0 j: 30 points:  45
i: 0 j: 40 points:  51
i: 1 j: 0 points:  1
i: 1 j: 10 points:  38
i: 1 j: 20 points:  48
i: 1 j: 30 points:  56
i: 1 j: 40 points:  61
i: 1 j: 50 points:  54
i: 1 j: 60 points:  57
i: 1 j: 70 points:  53
i: 1 j: 80 points:  57
i: 1 j: 90 points:  52
i: 1 j: 100 points:  57
i: 1 j: 110 points:  57
i: 1 j: 120 points:  57
i: 1 j: 130 points:  61
i: 1 j: 140 points:  63
i: 1 j: 150 points:  65
i: 1 j: 160 points:  65
i: 1 j: 170 points:  58
i: 1 j: 180 points:  58
i: 1 j: 190 points:  64
i: 1 j: 200 points:  67
i: 1 j: 210 points:  68
i: 1 j: 220 points:  73
i: 1 j: 230 points:  82
i: 1 j: 240 points:  86
i: 1 j: 250 points:  90
i: 1 j: 260 points:  91
i: 1 j: 270 points:  90
i: 1 j: 280 points:  90
i: 1 j: 290 points:  97
i: 1 j: 300 points:  94
i: 1 j: 310 points:  104
i: 1 j: 320 points:  107
i: 1 j: 330 points:  103
i: 1 j: 340 points:  101
i: 1 j: 350 points:  98
i: 1 j: 360 point

i: 4 j: 1570 points:  583
i: 4 j: 1580 points:  585
i: 4 j: 1590 points:  589
i: 4 j: 1600 points:  591
i: 4 j: 1610 points:  596
i: 4 j: 1620 points:  596
i: 4 j: 1630 points:  600
i: 4 j: 1640 points:  601
i: 4 j: 1650 points:  600
i: 4 j: 1660 points:  603
i: 4 j: 1670 points:  602
i: 4 j: 1680 points:  604
i: 4 j: 1690 points:  606
i: 4 j: 1700 points:  610
i: 4 j: 1710 points:  608
i: 4 j: 1720 points:  608
i: 4 j: 1730 points:  617
i: 4 j: 1740 points:  622
i: 4 j: 1750 points:  625
i: 4 j: 1760 points:  626
i: 4 j: 1770 points:  627
i: 4 j: 1780 points:  630
i: 4 j: 1790 points:  629
i: 4 j: 1800 points:  630
i: 4 j: 1810 points:  631
i: 4 j: 1820 points:  633
i: 4 j: 1830 points:  634
i: 4 j: 1840 points:  630
i: 4 j: 1850 points:  638
i: 4 j: 1860 points:  634
i: 4 j: 1870 points:  635
i: 4 j: 1880 points:  634
i: 4 j: 1890 points:  631
i: 4 j: 1900 points:  623
i: 4 j: 1910 points:  617
i: 4 j: 1920 points:  616
i: 4 j: 1930 points:  616
i: 4 j: 1940 points:  613
i: 4 j: 1950

i: 7 j: 500 points:  268
i: 7 j: 510 points:  270
i: 7 j: 520 points:  270
i: 7 j: 530 points:  278
i: 7 j: 540 points:  273
i: 7 j: 550 points:  272
i: 7 j: 560 points:  276
i: 7 j: 570 points:  271
i: 7 j: 580 points:  265
i: 7 j: 590 points:  265
i: 7 j: 600 points:  268
i: 7 j: 610 points:  264
i: 7 j: 620 points:  263
i: 7 j: 630 points:  263
i: 7 j: 640 points:  269
i: 7 j: 650 points:  279
i: 7 j: 660 points:  284
i: 7 j: 670 points:  290
i: 7 j: 680 points:  290
i: 7 j: 690 points:  293
i: 7 j: 700 points:  288
i: 7 j: 710 points:  289
i: 7 j: 720 points:  288
i: 7 j: 730 points:  301
i: 7 j: 740 points:  297
i: 8 j: 0 points:  1
i: 8 j: 10 points:  60
i: 8 j: 20 points:  78
i: 8 j: 30 points:  86
i: 8 j: 40 points:  94
i: 8 j: 50 points:  101
i: 8 j: 60 points:  102
i: 8 j: 70 points:  108
i: 8 j: 80 points:  108
i: 8 j: 90 points:  110
i: 8 j: 100 points:  114
i: 8 j: 110 points:  118
i: 8 j: 120 points:  133
i: 8 j: 130 points:  138
i: 8 j: 140 points:  147
i: 8 j: 150 point

i: 13 j: 530 points:  286
i: 13 j: 540 points:  288
i: 13 j: 550 points:  284
i: 13 j: 560 points:  278
i: 13 j: 570 points:  279
i: 13 j: 580 points:  276
i: 13 j: 590 points:  276
i: 13 j: 600 points:  284
i: 13 j: 610 points:  288
i: 13 j: 620 points:  288
i: 13 j: 630 points:  295
i: 13 j: 640 points:  299
i: 13 j: 650 points:  299
i: 13 j: 660 points:  305
i: 13 j: 670 points:  306
i: 13 j: 680 points:  307
i: 13 j: 690 points:  305
i: 13 j: 700 points:  303
i: 13 j: 710 points:  307
i: 13 j: 720 points:  316
i: 13 j: 730 points:  317
i: 13 j: 740 points:  324
i: 13 j: 750 points:  326
i: 13 j: 760 points:  327
i: 13 j: 770 points:  336
i: 13 j: 780 points:  335
i: 13 j: 790 points:  334
i: 13 j: 800 points:  334
i: 13 j: 810 points:  335
i: 13 j: 820 points:  342
i: 13 j: 830 points:  346
i: 13 j: 840 points:  348
i: 13 j: 850 points:  350
i: 13 j: 860 points:  359
i: 13 j: 870 points:  365
i: 13 j: 880 points:  377
i: 13 j: 890 points:  383
i: 13 j: 900 points:  381
i: 13 j: 910

i: 17 j: 990 points:  456
i: 17 j: 1000 points:  459
i: 17 j: 1010 points:  454
i: 17 j: 1020 points:  452
i: 17 j: 1030 points:  450
i: 17 j: 1040 points:  444
i: 17 j: 1050 points:  446
i: 17 j: 1060 points:  454
i: 17 j: 1070 points:  451
i: 17 j: 1080 points:  464
i: 17 j: 1090 points:  465
i: 17 j: 1100 points:  465
i: 17 j: 1110 points:  465
i: 17 j: 1120 points:  467
i: 17 j: 1130 points:  470
i: 17 j: 1140 points:  480
i: 17 j: 1150 points:  482
i: 17 j: 1160 points:  482
i: 17 j: 1170 points:  482
i: 17 j: 1180 points:  487
i: 17 j: 1190 points:  484
i: 17 j: 1200 points:  480
i: 17 j: 1210 points:  478
i: 17 j: 1220 points:  482
i: 17 j: 1230 points:  477
i: 17 j: 1240 points:  472
i: 17 j: 1250 points:  467
i: 17 j: 1260 points:  474
i: 18 j: 0 points:  1
i: 18 j: 10 points:  51
i: 18 j: 20 points:  61
i: 18 j: 30 points:  51
i: 18 j: 40 points:  51
i: 18 j: 50 points:  50
i: 18 j: 60 points:  56
i: 18 j: 70 points:  62
i: 18 j: 80 points:  52
i: 18 j: 90 points:  48
i: 18 j

...

i: 400 j: 10290 points:  610
i: 400 j: 10300 points:  615
i: 400 j: 10310 points:  611
i: 400 j: 10320 points:  610
i: 400 j: 10330 points:  609
i: 400 j: 10340 points:  608
i: 400 j: 10350 points:  609
i: 400 j: 10360 points:  611
i: 400 j: 10370 points:  611
i: 400 j: 10380 points:  606
i: 400 j: 10390 points:  609
i: 400 j: 10400 points:  608
i: 400 j: 10410 points:  607
i: 400 j: 10420 points:  608
i: 400 j: 10430 points:  605
i: 400 j: 10440 points:  601
i: 400 j: 10450 points:  598
i: 400 j: 10460 points:  594
i: 400 j: 10470 points:  598
i: 400 j: 10480 points:  600
i: 400 j: 10490 points:  600
i: 400 j: 10500 points:  599
i: 400 j: 10510 points:  602
i: 400 j: 10520 points:  609
i: 400 j: 10530 points:  615
i: 400 j: 10540 points:  620
i: 400 j: 10550 points:  621
i: 400 j: 10560 points:  622
i: 400 j: 10570 points:  621
i: 400 j: 10580 points:  626
i: 400 j: 10590 points:  627
i: 400 j: 10600 points:  629
i: 400 j: 10610 points:  631
i: 400 j: 10620 points:  632
i: 400 j: 1063

i: 400 j: 13120 points:  602
i: 400 j: 13130 points:  603
i: 400 j: 13140 points:  603
i: 400 j: 13150 points:  598
i: 400 j: 13160 points:  594
i: 400 j: 13170 points:  594
i: 400 j: 13180 points:  600
i: 400 j: 13190 points:  600
i: 400 j: 13200 points:  603
i: 400 j: 13210 points:  607
i: 400 j: 13220 points:  604
i: 400 j: 13230 points:  606
i: 400 j: 13240 points:  607
i: 400 j: 13250 points:  610
i: 400 j: 13260 points:  608
i: 400 j: 13270 points:  607
i: 400 j: 13280 points:  607
i: 400 j: 13290 points:  607
i: 400 j: 13300 points:  606
i: 400 j: 13310 points:  604
i: 400 j: 13320 points:  600
i: 400 j: 13330 points:  599
i: 400 j: 13340 points:  598
i: 400 j: 13350 points:  600
i: 400 j: 13360 points:  597
i: 400 j: 13370 points:  599
i: 400 j: 13380 points:  598
i: 400 j: 13390 points:  598
i: 400 j: 13400 points:  595
i: 400 j: 13410 points:  595
i: 400 j: 13420 points:  594
i: 400 j: 13430 points:  597
i: 400 j: 13440 points:  598
i: 400 j: 13450 points:  600
i: 400 j: 1346

i: 400 j: 15960 points:  555
i: 400 j: 15970 points:  550
i: 400 j: 15980 points:  552
i: 400 j: 15990 points:  555
i: 400 j: 16000 points:  557
i: 400 j: 16010 points:  557
i: 400 j: 16020 points:  553
i: 400 j: 16030 points:  544
i: 400 j: 16040 points:  534
i: 400 j: 16050 points:  540
i: 400 j: 16060 points:  541
i: 400 j: 16070 points:  544
i: 400 j: 16080 points:  549
i: 400 j: 16090 points:  550
i: 400 j: 16100 points:  552
i: 400 j: 16110 points:  553
i: 400 j: 16120 points:  556
i: 400 j: 16130 points:  556
i: 400 j: 16140 points:  558
i: 400 j: 16150 points:  558
i: 400 j: 16160 points:  560
i: 400 j: 16170 points:  559
i: 400 j: 16180 points:  556
i: 400 j: 16190 points:  550
i: 400 j: 16200 points:  548
i: 400 j: 16210 points:  545
i: 400 j: 16220 points:  544
i: 400 j: 16230 points:  544
i: 400 j: 16240 points:  545
i: 400 j: 16250 points:  545
i: 400 j: 16260 points:  542
i: 400 j: 16270 points:  541
i: 400 j: 16280 points:  542
i: 400 j: 16290 points:  538
i: 400 j: 1630

i: 400 j: 18790 points:  520
i: 400 j: 18800 points:  515
i: 400 j: 18810 points:  518
i: 400 j: 18820 points:  523
i: 400 j: 18830 points:  528
i: 400 j: 18840 points:  532
i: 400 j: 18850 points:  531
i: 400 j: 18860 points:  533
i: 400 j: 18870 points:  534
i: 400 j: 18880 points:  531
i: 400 j: 18890 points:  532
i: 400 j: 18900 points:  529
i: 400 j: 18910 points:  533
i: 400 j: 18920 points:  540
i: 400 j: 18930 points:  538
i: 400 j: 18940 points:  538
i: 400 j: 18950 points:  535
i: 400 j: 18960 points:  530
i: 400 j: 18970 points:  536
i: 400 j: 18980 points:  535
i: 400 j: 18990 points:  536
i: 400 j: 19000 points:  536
i: 400 j: 19010 points:  538
i: 400 j: 19020 points:  542
i: 400 j: 19030 points:  546
i: 400 j: 19040 points:  548
i: 400 j: 19050 points:  552
i: 400 j: 19060 points:  555
i: 400 j: 19070 points:  554
i: 400 j: 19080 points:  550
i: 400 j: 19090 points:  549
i: 400 j: 19100 points:  546
i: 400 j: 19110 points:  544
i: 400 j: 19120 points:  539
i: 400 j: 1913

i: 400 j: 21620 points:  671
i: 400 j: 21630 points:  674
i: 400 j: 21640 points:  678
i: 400 j: 21650 points:  680
i: 400 j: 21660 points:  684
i: 400 j: 21670 points:  689
i: 400 j: 21680 points:  690
i: 400 j: 21690 points:  686
i: 400 j: 21700 points:  688
i: 400 j: 21710 points:  686
i: 400 j: 21720 points:  692
i: 400 j: 21730 points:  689
i: 400 j: 21740 points:  686
i: 400 j: 21750 points:  686
i: 400 j: 21760 points:  686
i: 400 j: 21770 points:  686
i: 400 j: 21780 points:  685
i: 400 j: 21790 points:  688
i: 400 j: 21800 points:  694
i: 400 j: 21810 points:  693
i: 400 j: 21820 points:  692
i: 400 j: 21830 points:  692
i: 400 j: 21840 points:  695
i: 400 j: 21850 points:  699
i: 400 j: 21860 points:  694
i: 400 j: 21870 points:  697
i: 400 j: 21880 points:  692
i: 400 j: 21890 points:  689
i: 400 j: 21900 points:  682
i: 400 j: 21910 points:  686
i: 400 j: 21920 points:  682
i: 400 j: 21930 points:  678
i: 400 j: 21940 points:  685
i: 400 j: 21950 points:  685
i: 400 j: 2196

In [23]:
print('Number of nodes', len(vessel_graph_cn.nodes))
print('Number of edges', len(vessel_graph_cn.edges))
print('Average degree', sum(dict(vessel_graph_cn.degree).values()) / len(vessel_graph_cn.nodes))

Number of nodes 1366
Number of edges 1365
Average degree 1.9985358711566619


In [28]:
model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_cn)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [25]:
%%time
data_graph = parametrize_graph(vessel_graph_cn, edt_img)

CPU times: total: 141 ms
Wall time: 140 ms


C:\Users\Alicja\Alicja\medyczna\RenalVasculatureReconstruction\sphere_method\identification.py:440: RuntimeWarning: invalid value encountered in divide
  v1 /= np.linalg.norm(v1)
C:\Users\Alicja\Alicja\medyczna\RenalVasculatureReconstruction\sphere_method\identification.py:442: RuntimeWarning: invalid value encountered in divide
  v2 /= np.linalg.norm(v2)


In [26]:
%%time
data_graph_cl = clean_data_graph(data_graph)

CPU times: total: 0 ns
Wall time: 4.27 ms


In [27]:
model_with_graph = draw_graph_on_model(scaled_mask, data_graph_cl)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()